In [199]:
import gurobipy as gp
from gurobipy import *
from gurobipy import GRB
import numpy as np
import random
import pandas as pd
import math
# import scipy.sparse as sp

# Classes

In [200]:
class Dia:
    def __init__(self, id: int, nombre: str):
        self.id = id
        self.nombre = nombre

    def __str__(self) -> str:
        return self.nombre
        # return self.id
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.nombre == __value.nombre
 
class Horario:
    def __init__(self, id: int, inicio: str, fin: str, turnos: [str]=[]):
        self.id = id
        self.inicio = inicio
        self.fin = fin
        self.turnos = turnos

    def __str__(self) -> str:
        return self.inicio + "-" + self.fin
        # return self.id
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.inicio == __value.inicio and self.fin == __value.fin

class Grupo:
    def __init__(self, id: int, anio: int, turno: str=None, carrera: str=None, particion: int=None, recurse: bool=False) -> None:
        self.id = id
        self.anio = anio
        self.turno = turno
        self.carrera = carrera
        self.particion = particion
        self.recurse = recurse

    def __str__(self) -> str:
        st = ""
        # st += str(self.id) + "_"
        st += str(self.anio)
        st += str(self.carrera) if self.carrera is not None else ""
        st += "REC" if self.recurse else ""
        st += str(self.particion) if self.particion is not None else ""
        st += str(self.turno) if self.turno is not None else ""
        return st

    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.id == __value.id and self.anio == __value.anio
    
class BloqueHorario:
    def __init__(self, dia: Dia, horario: Horario) -> None:
        self.dia = dia
        self.horario = horario
        
    def __str__(self) -> str:
        return str(self.dia) + "_" + str(self.horario)
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.dia == __value.dia and self.horario == __value.horario
    
    def id(self):
        return (self.dia.id, self.horario.id)

# ver disponibilidad
class Profesor:
    def __init__(self, id: int, nombre: str):
        self.id = id
        self.nombre = nombre
        self.no_disponible = []
        self.prioridades = []
        self.lista_materias = [] # lista_materias[i] = (nombre_materia[i]: str, max_grupos: int)

    def __str__(self) -> str:
        return self.nombre

    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.nombre == __value.nombre


class Materia:
    def __init__(self,
                 id: int,
                 nombre: str,
                 carga_horaria: int=None,
                 cantidad_dias: int=3,
                 grupos: [Grupo]=[],
                 profesores: [Profesor] = []
                 ) -> None:
        
        self.nombre = nombre
        self.id = id
        self.carga_horaria = carga_horaria  #C_m
        self.cantidad_dias = cantidad_dias  #D_m
        self.grupos = grupos
        self.profesores = profesores
        
    def __str__(self) -> str:
        # if self.grupo is not None:
        #     return self.nombre + " " + str(self.grupo)
        return self.nombre
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.id == __value.id
    
    # def anio(self):
    #     if self.grupo is not None:
    #         return self.grupo.anio

    # H_max
    def horas_max(self) -> int:
        return math.ceil(self.carga_horaria / self.cantidad_dias)
    
    # H_min
    def horas_min(self) -> int:
        return math.floor(self.carga_horaria / self.cantidad_dias)
    
    def turnos(self):
        turnos = []
        for g in self.grupos:
            if g.turno is not None and g.turno not in turnos:
                turnos.append(g.turno)
        return turnos
    

class MateriaElectiva(Materia):
    def __init__(self,
                 id: int,
                 nombre: str,
                 carga_horaria: int=None,
                 cantidad_dias: int=3,
                 grupos: [Grupo]=[],
                 profesores: [Profesor]=[]
                 ) -> None:
        super().__init__(id, nombre, carga_horaria, cantidad_dias, grupos, profesores)

    
class Prioridad:
    def __init__(self, value: int, profesor: Profesor, bloque_horario: BloqueHorario) -> None:
        self.value = value
        self.profesor = profesor
        self.bloque_horario = bloque_horario
    
    def __str__(self) -> str:
        return str(self.profesor) + "_" + str(self.bloque_horario) + ":prioridad=" + str(self.value)
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.profesor == __value.profesor and self.bloque_horario == __value.bloque_horario
    
    def id(self):
        return (self.profesor.id, self.bloque_horario.id())
    
class Superposicion:
    def __init__(self, value: int, materia1: Materia, materia2: Materia) -> None:
        self.value = value
        self.materia1 = materia1
        self.materia2 = materia2
    
    def __str__(self) -> str:
        return "s_" + str(self.materia1) + "," + str(self.materia2) + "=" + str(self.value)
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and ((self.materia1 == __value.materia1 and self.materia2 == __value.materia2) or (self.materia1 == __value.materia2 and self.materia2 == __value.materia1))
    
    def id(self):
        return (self.profesor.id, self.bloque_horario.id())



In [201]:
# variables    
class u:
    def __init__(self, materia: Materia, horario: BloqueHorario) -> None:
        self.materia = materia
        self.horario = horario
        self.variable = None

    def __str__(self) -> str:
        return "u_{" + str(self.materia) + "_" + str(self.horario) + "}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.materia == __value.materia and self.horario == __value.horario
    
class v:
    def __init__(self, materia: Materia, dia: Dia) -> None:
        self.materia = materia
        self.dia = dia
        self.variable = None

    def __str__(self) -> str:
        return "v_{" + str(self.materia) + "_" + str(self.dia) + "}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.materia == __value.materia and self.dia == __value.dia

class w:
    def __init__(self, materia: Materia, profesor: Profesor) -> None:
        self.materia = materia
        self.profesor = profesor
        self.variable = None
    
    def __str__(self) -> str:
        return "w_{" + str(self.materia) + "_" + str(self.profesor) + "}"
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == type(self) and self.profesor == __value.profesor and self.materia == __value.materia
    
    

# Functions

In [202]:
# cargar datos

# crear materia
def add_materia(materias, nombre, carga_horaria, cantidad_dias, electiva: bool, grupos=[], profesores=[]):
    id = len(materias)
    if electiva:
        materias.append(MateriaElectiva(id, nombre, carga_horaria=carga_horaria, cantidad_dias=cantidad_dias,
                            grupos = grupos, profesores=profesores))
    else:
        materias.append(Materia(id, nombre, carga_horaria=carga_horaria, cantidad_dias=cantidad_dias,
                            grupos = grupos, profesores=profesores))

# crear grupo
def add_grupo(grupos, anio, turno=None, carrera=None, particion=None, recurse=False):
    id = len(grupos)
    grupos.append(Grupo(id, int(anio), turno, carrera, particion, recurse))

# crear profesor
def add_profesor(profesores, nombre):
    id = len(profesores)
    profesor = Profesor(id, nombre)
    if not profesor in profesores:
        profesores.append(profesor)

def lista_profesores(profesores, nombres):
    profs = []
    for n in nombres:
        for p in profesores:
            if str(p) == n:
                profs.append(p)
                break
    return profs

def lista_grupos(grupos, nombres):
    gs = []
    for n in nombres:
        for g in grupos:
            if str(g) == n:
                gs.append(g)
                break
    return gs


In [203]:
#prioridades
def update_no_disp(profesor, bloques_horario, no_disp_index):
    for i in no_disp_index:
        if bloques_horario[i] not in profesor.no_disponible:
            profesor.no_disponible.append(bloques_horario[i])

def update_prioridad(profesor, bloques_horario, array_prioridad):
    # array_prioridad[i] = [(d,h),a]
    for i in array_prioridad:
        b_id = i[0]
        value = i[1]
    
        if value == 0:
            update_no_disp(profesor, bloques_horario, [b_id])
        
        prior = Prioridad(value, profesor, bloques_horario[b_id])
        if not (prior in profesor.prioridades):
            profesor.prioridades.append(prior)

def random_pr(bloques_horario):
    pr_array = []
    for b in bloques_horario:
        pr_array.append([b, random.randint(0,3)])
        
    return pr_array

def promedio_prioridad(profesor, u_dict):
    return 0
    


In [204]:
#superposicion

def calcular_super(m1, m2):
    if m1 == m2 or isinstance(m1, MateriaElectiva) or isinstance(m2, MateriaElectiva):
        return Superposicion(0, m1, m2)
    else:
        s = False
        for g1 in m1.grupos:
            if g1 in m2.grupos:
                s = True
                break
        return Superposicion(1 if s else 0, m1, m2)

def calcular_super_electiva(m1, m2):
    if m1 == m2 or (not isinstance(m1, MateriaElectiva) and not isinstance(m2, MateriaElectiva)):
        return Superposicion(0, m1, m2)
    else:
        s = False
        for g1 in m1.grupos:
            if g1 in m2.grupos:
                s = True
                break
        return Superposicion(1 if s else 0, m1, m2)

In [205]:
#imprimir
def print_timetable(dias, horarios, u_dict, w_dict, grupos, anios):
    for a in anios:
        print('\n', '-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -')
        print("Año: ", str(a))
        # filtrar grupos dentro del año
        for g in grupos_anio(grupos, a):
            print('\n', "Grupo: ", str(g))
            print('\t', *[str(d) for d in dias], sep='\t\t')
            print('··········································································································')
            for h in horarios:
                mats = [search_materia(BloqueHorario(d,h), g, u_dict) for d in dias]
                lista = []
                for m in mats:
                    if m is None:
                        lista.append("---")
                    else:
                        # p = search_profesor(m, w_dict)
                        # lista.append(m.nombre + " " + str(p))
                        lista.append(str(m))
                print(str(h), *lista, sep='\t\t')
        # mostrar electivas por separado

def print_prof_timetable(dias, horarios, u_dict, w_dict, profesores):
    for p in profesores:
        print('\n', "Profesor: ", str(p))
        print('\t', *[str(d) for d in dias], sep='\t')
        print('·····················································')
        for h in horarios:
            mats = [search_materia_prof(BloqueHorario(d,h), p, u_dict, w_dict) for d in dias]
            lista = []
            for m in mats:
                if m is None:
                    lista.append("---")
                else:
                    lista.append(str(m))
            print(str(h), *lista, sep='\t')

def print_prioridades(dias, horarios, profesores):
    for p in profesores:
        print('\n', "Profesor: ", str(p))
        print('\t', *[str(d) for d in dias], sep='\t')
        print('·····················································')
        for h in horarios:
            prioridades = [str(search_prioridad(BloqueHorario(d,h),p)) for d in dias]
            print(str(h), *prioridades, sep='\t')

def print_prioridades_materia(dias, horarios, materia):
    print("Materia: ", str(materia))    
    print_prioridades(dias, horarios, materia.profesores)

def search_materia(b, g, u_dict):
    for u_id in u_dict:
        u_obj = u_dict[u_id]
        if u_obj.horario == b and g in u_obj.materia.grupos and round(u_obj.variable.x) == 1:
            return u_obj.materia
    return None

def search_profesor(materia, w_dict):
    if materia is None:
        return None
    for p in materia.profesores:
        if round(w_dict[materia.id, p.id].variable.x) == 1:
            return p
    return None

def search_profesor_by_nombre(profesores, nombre):
    for p in profesores:
        if p.nombre == nombre:
            return p
    return None

def search_materias_by_nombre(materias, nombre):
    ret = []
    for m in materias:
        if m.nombre == nombre:
            ret.append(m)
    return ret
    
def search_materia_prof(b, p, u_dict, w_dict):
    for u_id in u_dict:
        u_obj = u_dict[u_id]
        bloque = u_obj.horario
        profesor = search_profesor(u_obj.materia, w_dict)
        if bloque == b and profesor == p and round(u_obj.variable.x) == 1:
            return u_obj.materia
    return None

def search_prioridad(b, p):
    if b in p.no_disponible:
        return "-"
    for pr in p.prioridades:
        if pr.bloque_horario == b:
            return pr.value
        
def grupos_anio(grupos, anio):
    gs = []
    for g in grupos:
        if g.anio == anio:
            gs.append(g)
    return gs


In [206]:
#buscar materias

def materias_profesor(profesor, materias_total):
    mats = []

    for m in materias_total:
        if profesor in m.profesores:
            mats.append(m)

    return mats

def agrupar_materias(lista_materias):
    lista_nombres = {}

    for m in lista_materias:
        if str(m) not in lista_nombres:
            lista_nombres[str(m)] = [m]
        else:
            lista_nombres[str(m)].append(m)

    return lista_nombres

def materias_grupo(grupo, materias_total):
    materias = []

    for m in materias_total:
        if grupo is not None and grupo in m.grupos:
            materias.append(m)
            
    return materias


# Formulacion

## (1) Constantes / Datos

In [207]:
dias = [
    Dia(1, "lun"),
    Dia(2, "mar"),
    Dia(3, "mie"),
    Dia(4, "jue"),
    Dia(5, "vie")
]
dias_ids = []
for d in dias:
    dias_ids.append(d.id)

horarios = [
    Horario(1, "8:00", "8:50", ["m1", "m2"]),
    Horario(2, "8:50", "9:40", ["m1", "m2"]),
    Horario(3, "9:50", "10:40", ["m1", "m2"]),
    Horario(4, "10:40", "11:30", ["m1", "m2"]),
    Horario(5, "11:40", "12:30", ["m1", "m2"]),
    Horario(6, "12:30", "13:20", ["m1", "m2"]),
    Horario(7, "13:30", "14:20", ["m2"]),
    Horario(8, "13:10", "14:00", ["t1"]),
    Horario(9, "14:00", "14:50", ["t1", "t2"]),
    Horario(10, "15:00", "15:50", ["t1", "t2"]),
    Horario(11, "15:50", "16:40", ["t1", "t2"]),
    Horario(12, "16:50", "17:40", ["t1", "t2"]),
    Horario(13, "17:40", "18:30", ["t1", "t2"]),
    Horario(14, "18:40", "19:20", ["t1", "t2"])
]

horarios_ids = []
for h in horarios:
    horarios_ids.append(h.id)

bloques_horario = {}
for dia in dias:
    for horario in horarios:
        bloques_horario[(dia.id, horario.id)] = BloqueHorario(dia, horario)

bloques_horario_ids = []
for b in bloques_horario:
    bloques_horario_ids.append(b)


### Profesores

In [208]:
profesores = []

df = pd.read_excel("input.xlsm", sheet_name="profesores")

for i in range(len(df)):
    nombre = df["Profesor"][i]
    # # grupos_max = df["Max grupos"][i]
    # if not isinstance(grupos_max, int):
    #     grupos_max = None
    if isinstance(nombre, str):
        add_profesor(profesores, nombre)

profesores_ids = []
for p in profesores:
    profesores_ids.append(p.id)



c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [209]:
# prioridades random

# for p in profesores:
#     update_prioridad(p, bloques_horario, random_pr(bloques_horario))

In [210]:
# prioridades desde excel: unica hoja

# df = pd.read_excel("input.xlsm", sheet_name="prioridades")



In [211]:
# prioridades desde excel: 1 profesor por hoja

for prof in profesores:
    
    try:
        df = pd.read_excel("input.xlsm", sheet_name=str(prof))
    except:
        update_prioridad(prof, bloques_horario, random_pr(bloques_horario))
        continue

    array_prioridad = []
    # array_prioridad[i] = [(d,h),a]

    h_id = []

    for heading in df:

        if heading == str(prof):
            for row in df[heading]:
                h_id.append(row)
        else:
            i = 0
            for row in df[heading]:
                array_prioridad.append([(heading, h_id[i]), row])
                i += 1

    # print(array_prioridad)
    update_prioridad(prof, bloques_horario, array_prioridad)


### Grupos

In [212]:
#grupos
grupos = []

df = pd.read_excel("input.xlsm", sheet_name="grupos")

for i in range(len(df)):
    
    anio = int(df["Año"][i])
    turno = df["Turno"][i]
    carrera = df["Carrera"][i]
    particion = df["Particion"][i]
    recurse = isinstance(df["Recurse"][i], str)

    if str(particion) == "nan":
        particion = None
    else:
        particion = int(particion)

    if str(carrera) == "nan":
        carrera = None
    
    
    add_grupo(grupos, anio, turno, carrera, particion, recurse)

grupos_ids = []
for g in grupos:
    grupos_ids.append(g.id)


In [213]:
print([str(g) for g in grupos])

['11m1', '12m1', '13m1', '14t1', '1RECt1']


In [214]:
anios = []
for g in grupos:
    try:
        if g.anio not in anios:
            anios.append(g.anio)
    except:
        continue

### Materias 

In [215]:
#materias
materias = []

df = pd.read_excel("input.xlsm", sheet_name="materias")

for i in range(len(df)):
    nombre = df["Materia"][i]
    if not isinstance(nombre, str):
        continue
    carga_horaria = df["Carga Horaria"][i]
    cantidad_dias = df["Días por semana"][i]
    electiva = isinstance(df["Electiva"][i], str)

    profs = []
    p = 1
    while True:
        try:
            new_p = df["Profesor " + str(p)][i]
            if isinstance(new_p, str):
                profs.append(new_p)
            p += 1
        except:
            break

    gs = []
    g = 1
    while True:
        try:
            new_g = df["Grupo " + str(g)][i]
            if isinstance(new_g, str):
                gs.append(new_g)
            g += 1
        except:
            break
    
    # gs = lista_grupos(grupos, [x.strip() for x in str(df["Grupos"][i]).split(',')])
    # profs = lista_profesores(profesores, [x.strip() for x in str(df["Profesores"][i]).split(',')])

    add_materia(materias, nombre, carga_horaria, cantidad_dias, electiva, lista_grupos(grupos, gs), lista_profesores(profesores, profs))



c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [216]:
print_prioridades_materia(dias, horarios, materias[2])

Materia:  AL

 Profesor:  ap
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	1	1	1	1	1
8:50-9:40	1	1	1	1	1
9:50-10:40	1	1	1	1	1
10:40-11:30	-	-	-	-	-
11:40-12:30	-	-	-	-	-
12:30-13:20	-	-	-	-	-
13:30-14:20	-	-	-	-	-
13:10-14:00	-	-	-	-	-
14:00-14:50	-	-	-	-	-
15:00-15:50	-	-	-	-	-
15:50-16:40	-	-	-	-	-
16:50-17:40	-	-	-	-	-
17:40-18:30	-	-	-	-	-
18:40-19:20	-	-	-	-	-


In [217]:
for m in materias:
    print(str(m), ":", [str(g) for g in m.grupos])

AL : ['11m1']
AL : ['12m1']
AL : ['13m1']
AM1 : ['11m1']
AM1 : ['12m1']
AM1 : ['13m1']
PROG : ['11m1']
PROG : ['12m1']
PROG : ['13m1']
tING : ['11m1']
tING : ['12m1']
tING : ['13m1']
tINGb : ['11m1']
tINGb : ['12m1']
tINGb : ['13m1']
ALp : ['11m1']
ALp : ['12m1']
ALp : ['13m1']
AM1p : ['11m1']
AM1p : ['12m1']
AM1p : ['13m1']
PROGp : ['11m1']
PROGp : ['12m1']
PROGp : ['13m1']
AL : ['14t1']
AM1 : ['14t1']
PROG : ['14t1']
tING : ['14t1']
tINGb : ['14t1']
ALp : ['14t1']
AM1p : ['14t1']
PROGp : ['14t1']
ALN : ['1RECt1']
ALNp : ['1RECt1']
AM2 : ['1RECt1']
AM2p : ['1RECt1']
Fisica : ['1RECt1']


In [218]:
materias_ids = []
# materias_dict = {}
for m in materias:
    materias_ids.append(m.id)
    # materias_dict[m.id] = m


In [219]:
# materias por profesor:
df = pd.read_excel("input.xlsm", sheet_name="profesores")

for i in range(len(df)):
    nombre = df["Profesor"][i]

    lista_mats = []
    m = 1
    while True:
        try:
            new_m = df["Materia " + str(m)][i]
            
            if isinstance(new_m, str):
                grupos_max = int(df["Grupos materia " + str(m)][i])
                lista_mats.append({"nombre_materia" : new_m, "grupos_max" : grupos_max})
            m += 1
        except:
            break
    
    prof = search_profesor_by_nombre(profesores, nombre)
    if not prof is None:
        prof.lista_materias = lista_mats


    # add_materia(materias, nombre, carga_horaria, cantidad_dias, electiva, lista_grupos(grupos, gs), lista_profesores(profesores, profs))



c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [220]:
print(profesores[8].lista_materias)

[{'nombre_materia': 'PROG', 'grupos_max': 2}, {'nombre_materia': 'PROGp', 'grupos_max': 1}]


In [221]:
# superposicion
superposicion = {}
superposicion_electivas = {}

for m1 in materias:
    for m2 in materias:
        superposicion[(m1.id, m2.id)] = calcular_super(m1, m2)
        superposicion_electivas[(m1.id, m2.id)] = calcular_super_electiva(m1, m2)


## (2) Variables


$$ u_{mb} = [0,1] $$

    m: materia
    b: bloque horario = [dh]: dia y hora

$$ v_{md} = [0,1] $$

    m: materia
    d: dia

$$ w_{mp} = [0,1] $$

    m: materia
    p: profesor


In [222]:
# variables

u_dict = {}
for m in materias:   
    for b_id in bloques_horario:
        u_dict[(m.id, b_id)] = u(m, bloques_horario[b_id])
print("u: ", len(u_dict))

# print([str(us) for us in u_dict])

v_dict = {}
for m in materias:
    for d in dias:
        v_dict[(m.id, d.id)] = v(m, d)
print("v: ", len(v_dict))

w_dict = {}
for m in materias:
    for p in profesores:
        w_dict[(m.id, p.id)] = w(m, p)
print("w: ", len(w_dict))


u:  2590
v:  185
w:  629


In [251]:
print(len(materias))
print(len(bloques_horario))

37
70


In [223]:
# Create a new model
model = gp.Model("timetable")

In [224]:
# Create variables

# u_vars = m.addMVar(shape=len(u_dict), vtype=GRB.BINARY, name="u") # variable matrix

for u_i in u_dict: # crear variables "u" a partir de u_dict
    u_dict[u_i].variable = model.addVar(vtype=GRB.BINARY, name=str(u_dict[u_i]))

# print([ui.variable for ui in u_dict])

for v_i in v_dict: # crear variables "v" a partir de v_dict
    v_dict[v_i].variable = model.addVar(vtype=GRB.BINARY, name=str(v_dict[v_i]))

# v_vars = np.array([v_i.variable for v_i in v_dict])

# for r_i in r_dict:
#     r_dict[r_i].variable = model.addVar(vtype=GRB.CONTINUOUS, name=str(r_dict[r_i]))

for w_i in w_dict: # crear variables "v" a partir de v_dict
    w_dict[w_i].variable = model.addVar(vtype=GRB.BINARY, name=str(w_dict[w_i]))



## (3) Restricciones

### (3.1) Materias

#### (3.1.1) superposicion
$$ \sum_{m, m'}{u_{mb} \times S_{m,m'} \times u_{m'b}} = 0 $$

    para todo b

In [225]:
model.addConstrs(gp.quicksum(u_dict[m1, b].variable * superposicion[(m1, m2)].value * u_dict[m2, b].variable
                             for m1 in materias_ids for m2 in materias_ids)
                == 0 for b in bloques_horario_ids)

# for g in grupos:
#     mats = materias_grupo(g, materias)
#     if len(mats) > 1:
#         # model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)
#         model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)


{(1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (1, 3): <gurobi.QConstr Not Yet Added>,
 (1, 4): <gurobi.QConstr Not Yet Added>,
 (1, 5): <gurobi.QConstr Not Yet Added>,
 (1, 6): <gurobi.QConstr Not Yet Added>,
 (1, 7): <gurobi.QConstr Not Yet Added>,
 (1, 8): <gurobi.QConstr Not Yet Added>,
 (1, 9): <gurobi.QConstr Not Yet Added>,
 (1, 10): <gurobi.QConstr Not Yet Added>,
 (1, 11): <gurobi.QConstr Not Yet Added>,
 (1, 12): <gurobi.QConstr Not Yet Added>,
 (1, 13): <gurobi.QConstr Not Yet Added>,
 (1, 14): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (2, 3): <gurobi.QConstr Not Yet Added>,
 (2, 4): <gurobi.QConstr Not Yet Added>,
 (2, 5): <gurobi.QConstr Not Yet Added>,
 (2, 6): <gurobi.QConstr Not Yet Added>,
 (2, 7): <gurobi.QConstr Not Yet Added>,
 (2, 8): <gurobi.QConstr Not Yet Added>,
 (2, 9): <gurobi.QConstr Not Yet Added>,
 (2, 10): <gurobi.QConstr Not Yet Added>,
 (2, 11): 

#### (3.1.2) cubrir carga horaria para cada materia
$$ \sum_b{u_{mb}} = C_m $$

    para todo m (c_m: carga horaria)

In [226]:
for m in materias:
    model.addConstr(gp.quicksum(u_dict[m.id, b].variable for b in bloques_horario_ids) == m.carga_horaria)

#### (3.1.3) definicion de "v"

$$ v_{md} = {OR}_h [u_{mdh}] $$
    para todo m, d

In [227]:
# no es necesaria
# model.addConstrs(v_dict[(m, d)].variable == gp.or_(u_dict[(m, (d, h))].variable for h in horarios_ids) for m in materias_ids for d in dias_ids)

#### (3.1.4) particion de horas por materia

##### (3.1.4.1) fijar cantidad de dias por materia

$$ \sum_d {v_{md}} = D_m $$

    para todo m

In [228]:
for m in materias:
    model.addConstr(gp.quicksum(v_dict[m.id, d].variable for d in dias_ids) == m.cantidad_dias)

##### (3.1.4.2) fijar maximo y minimo de horas por dia

$$ v_{md} \times {H_{MIN}}_m \leq \sum_h {u_{mdh}} \leq v_{md} \times {H_{MAX}}_m $$

    para todo m, d

In [229]:
for m in materias:

    model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                     <= m.horas_max() * v_dict[m.id,d].variable for d in dias_ids)
    
    model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                     >= m.horas_min() * v_dict[m.id,d].variable for d in dias_ids)

##### (3.1.4.3) fijar materia a turno de horarios

$$ u_{mb} = 0 $$

    para todo b fuera de m.turno
    para todo m



In [230]:
for m in materias:
    # no_bloques_materia_ids = [i for i in bloques_horario if not bloques_horario[i].horario.turno in m.turnos()]
    no_bloques_materia_ids = []
    for b in bloques_horario:
        if len([t for t in bloques_horario[b].horario.turnos if t in m.turnos()]) == 0:
            no_bloques_materia_ids.append(b)
    model.addConstrs(u_dict[m.id, b].variable == 0 for b in no_bloques_materia_ids)


#### (3.1.5) horas consecutivas dentro de un dia

$$ u_{mdh} + \sum_h {u_{md(h)} \otimes u_{md(h+1)}} + u_{md(h_{max})} = 2 $$

$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} = 1 $$

$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} = v_{md} $$
    para todo m, d

In [231]:
model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 - gp.quicksum(u_dict[(m,(d,h))].variable * u_dict[(m,(d,h+1))].variable for h in horarios_ids[0:-1])
                 == v_dict[m, d].variable for m in materias_ids for d in dias_ids)

{(0, 1): <gurobi.QConstr Not Yet Added>,
 (0, 2): <gurobi.QConstr Not Yet Added>,
 (0, 3): <gurobi.QConstr Not Yet Added>,
 (0, 4): <gurobi.QConstr Not Yet Added>,
 (0, 5): <gurobi.QConstr Not Yet Added>,
 (1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (1, 3): <gurobi.QConstr Not Yet Added>,
 (1, 4): <gurobi.QConstr Not Yet Added>,
 (1, 5): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (2, 3): <gurobi.QConstr Not Yet Added>,
 (2, 4): <gurobi.QConstr Not Yet Added>,
 (2, 5): <gurobi.QConstr Not Yet Added>,
 (3, 1): <gurobi.QConstr Not Yet Added>,
 (3, 2): <gurobi.QConstr Not Yet Added>,
 (3, 3): <gurobi.QConstr Not Yet Added>,
 (3, 4): <gurobi.QConstr Not Yet Added>,
 (3, 5): <gurobi.QConstr Not Yet Added>,
 (4, 1): <gurobi.QConstr Not Yet Added>,
 (4, 2): <gurobi.QConstr Not Yet Added>,
 (4, 3): <gurobi.QConstr Not Yet Added>,
 (4, 4): <gurobi.QConstr Not Yet Added>,
 (4, 5): <gurobi

#### (3.1.6) evitar dias consecutivos para una misma materia

$$ \sum_d {v_{m(d)} · v_{m(d+1)}} = 0 $$

    para todo m

In [232]:
model.addConstrs(gp.quicksum(v_dict[m,d].variable * v_dict[m,d+1].variable for d in dias_ids[0:-1]) == 0
                 for m in materias_ids)


{0: <gurobi.QConstr Not Yet Added>,
 1: <gurobi.QConstr Not Yet Added>,
 2: <gurobi.QConstr Not Yet Added>,
 3: <gurobi.QConstr Not Yet Added>,
 4: <gurobi.QConstr Not Yet Added>,
 5: <gurobi.QConstr Not Yet Added>,
 6: <gurobi.QConstr Not Yet Added>,
 7: <gurobi.QConstr Not Yet Added>,
 8: <gurobi.QConstr Not Yet Added>,
 9: <gurobi.QConstr Not Yet Added>,
 10: <gurobi.QConstr Not Yet Added>,
 11: <gurobi.QConstr Not Yet Added>,
 12: <gurobi.QConstr Not Yet Added>,
 13: <gurobi.QConstr Not Yet Added>,
 14: <gurobi.QConstr Not Yet Added>,
 15: <gurobi.QConstr Not Yet Added>,
 16: <gurobi.QConstr Not Yet Added>,
 17: <gurobi.QConstr Not Yet Added>,
 18: <gurobi.QConstr Not Yet Added>,
 19: <gurobi.QConstr Not Yet Added>,
 20: <gurobi.QConstr Not Yet Added>,
 21: <gurobi.QConstr Not Yet Added>,
 22: <gurobi.QConstr Not Yet Added>,
 23: <gurobi.QConstr Not Yet Added>,
 24: <gurobi.QConstr Not Yet Added>,
 25: <gurobi.QConstr Not Yet Added>,
 26: <gurobi.QConstr Not Yet Added>,
 27: <gurob

### (3.2) Profesores

#### (3.2.1) indisponibilidad

$$ w_{mp} \times u_{mb} = 0 $$

    para todo p sin disponibilidad en b

In [233]:
for m in materias:
    for p in m.profesores:
        for b in p.no_disponible:
            model.addConstr(u_dict[m.id, b.id()].variable * w_dict[m.id, p.id].variable == 0)

#### (3.2.2) unica materia por profesor para un mismo bloque horario

$$ \sum_{m} {w_{mp} \times u_{mb}} \leq 1 $$

    para todo b, p

In [234]:
for p in profesores:
    model.addConstrs(gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids)
                     <= 1 for b in bloques_horario_ids)

#### (3.2.3) profesores por materia


##### (3.2.3.1) limitar profesores a lista

$$ w_{mp} = 0 $$

    para todo p not in m.lista_profesores
    para todo m

In [235]:
for m in materias:
    for p in profesores:
        if p not in m.profesores:
            model.addConstr(w_dict[m.id, p.id].variable == 0)

##### (3.2.3.2) unico profesor

$$ \sum_p {w_{mp}} = 1 $$

    para todo m

In [236]:
for m in materias:
    model.addConstr(gp.quicksum(w_dict[m.id, p].variable for p in profesores_ids) == 1)

#### (3.2.4) carga horaria por docente: limitar la cantidad de materias por profesor


$$ \sum_m {w_{mp}} = K_p $$

    para todo p



In [237]:
# separar por nombre de materia
for p in profesores:
    for l in p.lista_materias:
        mats = search_materias_by_nombre(materias, l["nombre_materia"])
        model.addConstr(gp.quicksum(w_dict[m.id, p.id].variable for m in mats) <= l["grupos_max"])
        

## (4) Funcion Objetivo

In [238]:
# Set objective
OBJ = gp.QuadExpr()

### (4.1) Prioridad horaria de los docentes

$$ MIN: \sum_{m,b,p} {A_{pb}·w_{mp}·u_{mb}} $$


In [239]:
OBJ1 = gp.QuadExpr()

count = 0
for m in materias:
    for p in profesores:
        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            OBJ1 += A * w_dict[m.id, p.id].variable * u_dict[m.id, b.id()].variable
            count += 1

OBJ += OBJ1

print(count)


44030


### (4.2) Disponibilidad de materias electivas

$$ MIN: \sum_{b,m,m'} {u_{mb}·SE_{m,m'}·u_{m'b}} $$


In [240]:
OBJ2 = gp.QuadExpr()

for b in bloques_horario_ids:
    for m1 in materias_ids:
        for m2 in materias_ids:
            SE = superposicion_electivas[(m1, m2)].value
            OBJ2 += u_dict[m1, b].variable * SE * u_dict[m2, b].variable

OBJ += 10 * OBJ2

## Resolver

In [241]:
model.setObjective(OBJ, GRB.MINIMIZE)

In [242]:
for p in profesores:
    print(str(p), p.lista_materias)

svv [{'nombre_materia': 'AL', 'grupos_max': 1}]
mp [{'nombre_materia': 'AL', 'grupos_max': 1}]
ap [{'nombre_materia': 'AL', 'grupos_max': 1}]
am [{'nombre_materia': 'AM1', 'grupos_max': 1}]
gc [{'nombre_materia': 'AM1', 'grupos_max': 1}, {'nombre_materia': 'AM1p', 'grupos_max': 3}, {'nombre_materia': 'AM2', 'grupos_max': 1}, {'nombre_materia': 'AM2p', 'grupos_max': 1}]
jd [{'nombre_materia': 'AM1', 'grupos_max': 1}]
kb [{'nombre_materia': 'PROG', 'grupos_max': 1}]
vt [{'nombre_materia': 'PROG', 'grupos_max': 1}, {'nombre_materia': 'PROGp', 'grupos_max': 1}]
jc [{'nombre_materia': 'PROG', 'grupos_max': 2}, {'nombre_materia': 'PROGp', 'grupos_max': 1}]
ad [{'nombre_materia': 'tING', 'grupos_max': 4}]
as [{'nombre_materia': 'tINGb', 'grupos_max': 4}]
jf [{'nombre_materia': 'ALp', 'grupos_max': 4}, {'nombre_materia': 'AM1p', 'grupos_max': 1}]
fm [{'nombre_materia': 'PROGp', 'grupos_max': 2}]
ds [{'nombre_materia': 'AL', 'grupos_max': 1}]
lg [{'nombre_materia': 'AM1', 'grupos_max': 1}]
jk [

In [243]:
model.setParam("TimeLimit", 60*60*2)
model.optimize()

# if model.Status == GRB.INFEASIBLE:
    # model.computeIIS()

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2495 rows, 3404 columns and 11025 nonzeros
Model fingerprint: 0xaae50b73
Model has 30710 quadratic objective terms
Model has 2816 quadratic constraints
Variable types: 0 continuous, 3404 integer (3404 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 1936 rows and 2657 columns
Presolve time: 0.12s
Presolved: 3121 rows, 1871 columns, 10515 nonzeros
Variable types: 0 continuous, 1871 integer (1871 binary)

Root relaxation: objective 9.000000e+01

In [244]:
print('Obj: %g' % model.ObjVal)
print('=============================================================')
print_timetable(dias, horarios, u_dict, w_dict, grupos, anios)


Obj: 116

 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
Año:  1

 Grupo:  11m1
			lun		mar		mie		jue		vie
··········································································································
8:00-8:50		AL		AM1p		---		PROGp		---
8:50-9:40		AL		AM1p		AL		PROGp		AM1
9:50-10:40		AL		AM1		AL		ALp		AM1
10:40-11:30		tING		AM1		PROG		ALp		AM1
11:40-12:30		tING		---		PROG		tINGb		---
12:30-13:20		tING		---		PROG		tINGb		---
13:30-14:20		---		---		---		---		---
13:10-14:00		---		---		---		---		---
14:00-14:50		---		---		---		---		---
15:00-15:50		---		---		---		---		---
15:50-16:40		---		---		---		---		---
16:50-17:40		---		---		---		---		---
17:40-18:30		---		---		---		---		---
18:40-19:20		---		---		---		---		---

 Grupo:  12m1
			lun		mar		mie		jue		vie
··········································································································
8:00-8:50		PROGp		AL		PROG		tING		---
8:50-9:40		PROGp		AL

In [245]:
# imprimir calendario para profesores
print_prof_timetable(dias, horarios, u_dict, w_dict, profesores)


 Profesor:  svv
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	---	AL	---	---	---
8:50-9:40	---	AL	---	---	---
9:50-10:40	---	---	---	---	---
10:40-11:30	---	---	---	AL	---
11:40-12:30	---	---	---	AL	---
12:30-13:20	---	---	---	AL	---
13:30-14:20	---	---	---	---	---
13:10-14:00	---	---	---	---	---
14:00-14:50	---	---	---	---	---
15:00-15:50	---	---	---	---	---
15:50-16:40	---	---	---	---	---
16:50-17:40	---	---	---	---	---
17:40-18:30	---	---	---	---	---
18:40-19:20	---	---	---	---	---

 Profesor:  mp
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	AL	---	---	---	---
8:50-9:40	AL	---	AL	---	---
9:50-10:40	AL	---	AL	---	---
10:40-11:30	---	---	---	---	---
11:40-12:30	---	---	---	---	---
12:30-13:20	---	---	---	---	---
13:30-14:20	---	---	---	---	---
13:10-14:00	---	---	---	---	---
14:00-14:50	---	---	---	---	---
15:00-15:50	---	---	---	---	---
15:50-16:40	---	---	---	---	---
16:50-17:40	---	---	---	---	---
17:

12:30-13:20	---	---	---	---	---
13:30-14:20	---	---	---	---	---
13:10-14:00	---	---	---	---	---
14:00-14:50	---	---	---	---	---
15:00-15:50	---	---	---	---	---
15:50-16:40	AL	---	---	---	---
16:50-17:40	AL	---	---	---	AL
17:40-18:30	---	---	---	---	AL
18:40-19:20	---	---	---	---	AL

 Profesor:  lg
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	---	---	---	---	---
8:50-9:40	---	---	---	---	---
9:50-10:40	---	---	---	---	---
10:40-11:30	---	---	---	---	---
11:40-12:30	---	---	---	---	---
12:30-13:20	---	---	---	---	---
13:30-14:20	---	---	---	---	---
13:10-14:00	AM1	---	---	---	---
14:00-14:50	AM1	---	AM1	---	---
15:00-15:50	AM1	---	AM1	---	---
15:50-16:40	---	---	---	---	---
16:50-17:40	---	---	---	---	---
17:40-18:30	---	---	---	---	---
18:40-19:20	---	---	---	---	---

 Profesor:  jk
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	---	---	---	---	---
8:50-9:40	---	---	---	---	---
9:50-10:40	---	---	---	---	---

In [246]:
for p in profesores:
    print(str(p), [str(m) for m in materias_profesor(p, materias)])

svv ['AL', 'AL']
mp ['AL', 'AL']
ap ['AL']
am ['AM1', 'AM1', 'AM1']
gc ['AM1', 'AM1', 'AM1', 'AM1p', 'AM1p', 'AM1p', 'AM2', 'AM2p']
jd ['AM1', 'AM1', 'AM1']
kb ['PROG', 'PROG', 'PROG']
vt ['PROG', 'PROG', 'PROG', 'PROGp', 'PROGp', 'PROGp']
jc ['PROG', 'PROG', 'PROG', 'PROG', 'PROGp']
ad ['tING', 'tING', 'tING', 'tING']
as ['tINGb', 'tINGb', 'tINGb', 'tINGb']
jf ['ALp', 'ALp', 'ALp', 'ALp', 'AM1p']
fm ['PROGp', 'PROGp', 'PROGp']
ds ['AL']
lg ['AM1']
jk ['ALN', 'ALNp']
jpf ['Fisica']


In [247]:
print_prioridades(dias, horarios, profesores)


 Profesor:  svv
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	3	1	1	1	3
8:50-9:40	3	1	1	1	3
9:50-10:40	3	1	1	1	3
10:40-11:30	3	1	1	1	3
11:40-12:30	3	1	1	1	3
12:30-13:20	3	1	1	1	3
13:30-14:20	3	1	1	1	3
13:10-14:00	-	-	-	-	-
14:00-14:50	-	-	-	-	-
15:00-15:50	-	-	-	-	-
15:50-16:40	-	-	-	-	-
16:50-17:40	-	-	-	-	-
17:40-18:30	-	-	-	-	-
18:40-19:20	-	-	-	-	-

 Profesor:  mp
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	1	1	1	1	1
8:50-9:40	1	1	1	1	1
9:50-10:40	1	1	1	1	1
10:40-11:30	1	1	1	1	1
11:40-12:30	1	1	1	1	1
12:30-13:20	1	1	1	1	1
13:30-14:20	1	1	1	1	1
13:10-14:00	1	1	1	1	1
14:00-14:50	1	1	1	1	1
15:00-15:50	1	1	1	1	1
15:50-16:40	1	1	1	1	1
16:50-17:40	1	1	1	1	1
17:40-18:30	1	1	1	1	1
18:40-19:20	1	1	1	1	1

 Profesor:  ap
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	1	1	1	1	1
8:50-9:40	1	1	1	1	1
9:50-10:40	1	1	1	1	1
10:40-11:30	-	-	-	-	-
11:40-12:30	-	-	-	-	-
12:30-13:20	

In [248]:
CARGA_HORARIA_TOTAL = gp.quicksum(m.carga_horaria for m in materias)

print("Promedio general de prioridad horaria: ", OBJ1.getValue()/CARGA_HORARIA_TOTAL.getValue())
# print("Casos de materias con dias consecutivos: ", int(OBJ2.getValue()/10))


Promedio general de prioridad horaria:  1.0175438596491229


In [249]:
# for u_i in u_dict:
#     u = u_dict[u_i]
#     print(str(u) + " = ", u.variable.X)

# for v_i in v_dict:
#     v = v_dict[v_i]
#     print(str(v) + " = ", v.variable.X)

# for w_i in w_dict:
#     w = w_dict[w_i]
#     print(str(w) + " = ", w.variable.X)


In [250]:
model.write('timetable_opt.lp')
model.write('timetable_opt.mps')

# model.display()